# Cleaning and Preprocessing

## 1. Notebook overview

This notebook prepares the dataset for modeling by performing data cleaning, feature engineering, and preprocessing.

Specifically, it:
- Reloads the cleaned dataset from the previous EDA step.
- Applies cleaning and formatting for consistency.
- Encodes categorical variables appropriately.
- Scales numerical features if needed.
- Handles class imbalance in the target variable.
- Splits the data into training and test sets.
- Exports the preprocessed dataset for modeling.

This is the second step in the pipeline following `01_eda.ipynb`, and it feeds into `03_modeling.ipynb`.

In [1]:
# Imports for preprocessing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Set options for display
pd.set_option('display.max_columns', None)

# Set random seed
RANDOM_STATE = 42

print("Preprocessing environment initialized.")

Preprocessing environment initialized.


## 2. Reload data and confirm schema

We load the cleaned dataset exported from the EDA notebook (`data_01.csv`) and verify its structure before proceeding with preprocessing.

This step ensures:
- The dataset was saved correctly.
- The schema matches expectations (column names, data types).
- There are no unexpected missing values or type mismatches introduced during export.

We also validate that the dataset includes exactly the expected columns — no more, no less. This prevents issues downstream if column names are altered, dropped, or duplicated.

The expected schema includes only meaningful, cleaned features after removing non-informative columns in the EDA phase.

In [2]:
# Load cleaned data
df = pd.read_csv('../data/processed/data_01.csv')

# Define expected column names after EDA cleanup
expected_columns = [
    'Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
    'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction',
    'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
    'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
    'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
    'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
    'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
    'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager'
]

# Get actual columns from the loaded DataFrame
actual_columns = list(df.columns)

# Compare against expected
missing_columns = set(expected_columns) - set(actual_columns)
unexpected_columns = set(actual_columns) - set(expected_columns)

# Display results
if not missing_columns and not unexpected_columns:
    print("Column schema validation passed.")
else:
    if missing_columns:
        print("Missing columns:", missing_columns)
    if unexpected_columns:
        print("Unexpected columns:", unexpected_columns)

Column schema validation passed.


## 3. Cleaning checks

We perform additional cleaning checks on the dataset before continuing preprocessing.

This includes:
- Verifying data types are appropriate.
- Checking for unexpected nulls (none should exist).
- Ensuring no constant or identifier columns remain.
- Confirming target class distribution.

In [3]:
# Check for nulls (none expected)
null_counts = df.isnull().sum()
if null_counts.any():
    print("Unexpected null values found:")
    display(null_counts[null_counts > 0])
else:
    print("No null values found.")

# Recheck data types
print("\nData types:")
display(df.dtypes)

# Identify constant columns (only one unique value)
nunique = df.nunique()
constant_cols = nunique[nunique == 1].index.tolist()

if constant_cols:
    print(f"Constant columns detected and dropped: {constant_cols}")
    df.drop(columns=constant_cols, inplace=True)
    print(f"New shape after dropping: {df.shape}")
else:
    print("No constant columns detected.")

# Confirm target variable distribution
print("\nClass balance in 'Attrition':")
display(df['Attrition'].value_counts(normalize=True).round(3))

No null values found.

Data types:


Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole  

No constant columns detected.

Class balance in 'Attrition':


Attrition
No     0.839
Yes    0.161
Name: proportion, dtype: float64

## 4. Feature engineering

In this section, we enhance the feature space with domain-informed transformations and interactions that may improve model performance.

We include:

### Tenure-related features
- `TenureCategory`: Bins `YearsAtCompany` into ranges to capture non-linear tenure patterns.
- `YearsAtCompanyRatio`: Fraction of career spent at current company.
- `YearsInRoleRatio`: Stability in the same role.
- `YearsWithManagerRatio`: Indicates long-term relationship with supervisor.

### Income-related features
- `IncomePerYear`: Ratio of monthly income to tenure.
- `HighIncomeShortTenure`: Flags early high earners (possible burnout risk).
- `HighIncomeFlag`: Binary flag for income above median.
- `HourlyRateBucket`: Bucketized hourly pay to reduce skew.
- `IncomePerEduLevel`: Normalized income by education level.
- `IncomeZScoreWithinEdu`: Z-score of income within each education level group.

### Job role and travel groupings
- `JobRoleGroup`: Simplifies rare job roles into broader categories.
- `Travel_Occupation`: Interaction between business travel frequency and job role.

### Satisfaction-based signals
- `SatisfactionAvg`: Mean satisfaction across 3 dimensions.
- `SatisfactionRange`: Range between highest and lowest satisfaction.
- `LowSatisfactionFlag`: Any satisfaction dimension rated ≤ 2.

### Interaction and composite flags
- `OverTime_JobLevel`: Combines overtime flag with job level.
- `StressRisk`: Composite risk index based on overtime, commute, and satisfaction.

These features are designed to expose nonlinear effects, interactions, and latent variables the model might otherwise miss.

In [4]:
# Tenure-based Features
df['TenureCategory'] = pd.cut(
    df['YearsAtCompany'],
    bins=[-1, 2, 5, 10, np.inf],
    labels=['<3 yrs', '3-5 yrs', '6-10 yrs', '10+ yrs']
)

df['YearsAtCompanyRatio'] = df['YearsAtCompany'] / df['TotalWorkingYears'].replace(0, np.nan) # Avoids division by 0 - nulls created here are handled in the next cell
df['YearsInRoleRatio'] = df['YearsInCurrentRole'] / df['YearsAtCompany'].replace(0, np.nan)
df['YearsWithManagerRatio'] = df['YearsWithCurrManager'] / df['YearsAtCompany'].replace(0, np.nan)

# Income-based Features
df['IncomePerYear'] = df['MonthlyIncome'] / df['YearsAtCompany'].replace(0, np.nan)
df['HighIncomeShortTenure'] = (
    (df['MonthlyIncome'] > df['MonthlyIncome'].median()) &
    (df['YearsAtCompany'] < 3)
).astype(int)
df['HighIncomeFlag'] = (df['MonthlyIncome'] > df['MonthlyIncome'].median()).astype(int)

df['HourlyRateBucket'] = pd.qcut(df['HourlyRate'], q=4, labels=False)
df['IncomePerEduLevel'] = df['MonthlyIncome'] / df['Education'].replace(0, np.nan)

df['IncomeZScoreWithinEdu'] = df.groupby('Education')['MonthlyIncome'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0) # Population standard deviation; population = all employees at each education level; avoids dividing by 0
)

# Job Role Groupings
df['JobRoleGroup'] = df['JobRole'].replace({
    'Laboratory Technician': 'Technical',
    'Research Scientist': 'Technical',
    'Healthcare Representative': 'Sales',
    'Sales Executive': 'Sales',
    'Sales Representative': 'Sales'
})

# Satisfaction Features
df['SatisfactionAvg'] = df[[
    'EnvironmentSatisfaction', 'JobSatisfaction', 'RelationshipSatisfaction'
]].mean(axis=1)

df['SatisfactionRange'] = df[[
    'EnvironmentSatisfaction', 'JobSatisfaction', 'RelationshipSatisfaction'
]].max(axis=1) - df[[
    'EnvironmentSatisfaction', 'JobSatisfaction', 'RelationshipSatisfaction'
]].min(axis=1)

df['LowSatisfactionFlag'] = (
    (df['EnvironmentSatisfaction'] <= 2) |
    (df['JobSatisfaction'] <= 2) |
    (df['RelationshipSatisfaction'] <= 2)
).astype(int)

# Interaction Features
df['OverTime_JobLevel'] = df['OverTime'].astype(str) + "_" + df['JobLevel'].astype(str)
df['Travel_Occupation'] = df['BusinessTravel'].astype(str) + "_" + df['JobRole'].astype(str)

# Composite Risk Flag
df['StressRisk'] = (
    (df['OverTime'] == 'Yes') &
    (df['DistanceFromHome'] > 10) &
    (df['SatisfactionAvg'] < 2.5)
).astype(int)

print("All engineered features added.")

All engineered features added.


#### Handle division-based nulls and flag zero-tenure employees

Several ratio features (`YearsInRoleRatio`, `YearsWithManagerRatio`, `IncomePerYear`, `YearsAtCompanyRatio`) use `YearsAtCompany` as a denominator. For a small number of employees where `YearsAtCompany == 0`, these ratios became `NaN` due to division by zero.

To preserve eliminate these nulls:

- All affected ratio columns are imputed with `0`, reflecting minimal tenure context (new hires, etc).
- A `ZeroCompanyTenureFlag` marks these edge cases for interpretability.
- A `NoWorkHistoryFlag` separately flags employees with no prior work experience (`TotalWorkingYears == 0`).

This ensures the dataset remains numerically stable while retaining signal from meaningful outliers.

In [ ]:
# Fill ratio-based nulls caused by YearsAtCompany == 0
df['YearsInRoleRatio'] = df['YearsInRoleRatio'].fillna(0)
df['YearsWithManagerRatio'] = df['YearsWithManagerRatio'].fillna(0)
df['IncomePerYear'] = df['IncomePerYear'].fillna(0)
df['YearsAtCompanyRatio'] = df['YearsAtCompanyRatio'].fillna(0)

# Flag structural edge cases for model interpretability
df['ZeroCompanyTenureFlag'] = (df['YearsAtCompany'] == 0).astype(int)
df['NoWorkHistoryFlag'] = ((df['YearsAtCompany'] == 0) & (df['TotalWorkingYears'] == 0)).astype(int)

YearsAtCompanyRatio
1.000000    463
0.500000     59
0.833333     55
0.000000     44
0.800000     41
           ... 
0.863636      1
0.722222      1
0.034483      1
0.680000      1
0.529412      1
Name: count, Length: 192, dtype: int64

## 5. Encoding categorical variables

To prepare the dataset for modeling, we encode categorical variables into numerical form.

We apply:
- **Label encoding** for binary and ordinal variables (`Attrition`, `TenureCategory`, ...)
- **One-hot encoding** for nominal variables with more than 2 categories

This transformation ensures that all features are numeric, as required by logistic regression. We also drop one dummy column per one-hot-encoded feature to avoid multicollinearity.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Age                       1470 non-null   int64   
 1   Attrition                 1470 non-null   object  
 2   BusinessTravel            1470 non-null   object  
 3   DailyRate                 1470 non-null   int64   
 4   Department                1470 non-null   object  
 5   DistanceFromHome          1470 non-null   int64   
 6   Education                 1470 non-null   int64   
 7   EducationField            1470 non-null   object  
 8   EnvironmentSatisfaction   1470 non-null   int64   
 9   Gender                    1470 non-null   object  
 10  HourlyRate                1470 non-null   int64   
 11  JobInvolvement            1470 non-null   int64   
 12  JobLevel                  1470 non-null   int64   
 13  JobRole                   1470 non-null   object

In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# Make a copy to work from
df_encoded = df.copy()

# Binary Label Encoding
binary_cols = [
    'Attrition', 'OverTime', 'Gender', 'HighIncomeFlag',
    'LowSatisfactionFlag', 'HighIncomeShortTenure', 'StressRisk'
]

for col in binary_cols:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

# Ordinal Encoding
df_encoded['TenureCategory'] = df_encoded['TenureCategory'].map({
    '<3 yrs': 0,
    '3-5 yrs': 1,
    '6-10 yrs': 2,
    '10+ yrs': 3
})

# One-hot Encoding
nominal_cols = [
    'BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus',
    'JobRoleGroup', 'Travel_Occupation', 'OverTime_JobLevel'
]

df_encoded = pd.get_dummies(df_encoded, columns=nominal_cols, drop_first=True) # Avoid multicollinearity

print("Categorical variables encoded.")
print(f"Encoded shape: {df_encoded.shape}")

# Inspect new columns
original_cols = set(df.columns)
encoded_cols = set(df_encoded.columns)
new_columns = sorted(encoded_cols - original_cols)

if new_columns:
    print(f"\n{len(new_columns)} new columns added via one-hot encoding:\n")
    for col in new_columns:
        print(f"  {col}")
else:
    print("No new columns were added via one-hot encoding.")

# Group one-hot columns by source
grouped_dummies = defaultdict(list)
for col in new_columns:
    for original in nominal_cols:
        if col.startswith(original + "_"):
            grouped_dummies[original].append(col)

# Display grouped dummy columns
for key in sorted(grouped_dummies):
    values = grouped_dummies[key]
    print(f"\n{key} → {len(values)} dummy columns:")
    for v in sorted(values):
        print(f"  {v}")

Categorical variables encoded.
Encoded shape: (1470, 101)

🆕 59 new columns added via one-hot encoding:

  BusinessTravel_Travel_Frequently
  BusinessTravel_Travel_Rarely
  Department_Research & Development
  Department_Sales
  EducationField_Life Sciences
  EducationField_Marketing
  EducationField_Medical
  EducationField_Other
  EducationField_Technical Degree
  JobRoleGroup_Manager
  JobRoleGroup_Manufacturing Director
  JobRoleGroup_Research Director
  JobRoleGroup_Sales
  JobRoleGroup_Technical
  JobRole_Human Resources
  JobRole_Laboratory Technician
  JobRole_Manager
  JobRole_Manufacturing Director
  JobRole_Research Director
  JobRole_Research Scientist
  JobRole_Sales Executive
  JobRole_Sales Representative
  MaritalStatus_Married
  MaritalStatus_Single
  OverTime_JobLevel_No_2
  OverTime_JobLevel_No_3
  OverTime_JobLevel_No_4
  OverTime_JobLevel_No_5
  OverTime_JobLevel_Yes_1
  OverTime_JobLevel_Yes_2
  OverTime_JobLevel_Yes_3
  OverTime_JobLevel_Yes_4
  OverTime_JobLevel_

### Removing rare dummy variables

Some one-hot encoded columns may represent categories with very few samples, which can introduce instability during training. We drop dummy variables that appear in fewer than 10 observations.

In [8]:
# Define minimum threshold
min_count = 10

# Identify dummy columns that were just created
original_cols = set(df.columns)
encoded_cols = set(df_encoded.columns)
new_dummies = sorted(encoded_cols - original_cols)

# Check frequency of each dummy column
rare_dummies = [col for col in new_dummies if df_encoded[col].sum() < min_count]

# Drop and report
if rare_dummies:
    df_encoded.drop(columns=rare_dummies, inplace=True)
    print(f"Dropped {len(rare_dummies)} rare dummy columns (fewer than {min_count} observations):")
    for col in rare_dummies:
        print(f"  {col}")
else:
    print("No rare dummy columns to drop.")

Dropped 3 rare dummy columns (fewer than 10 observations):
  Travel_Occupation_Non-Travel_Human Resources
  Travel_Occupation_Non-Travel_Research Director
  Travel_Occupation_Non-Travel_Sales Representative


## 6. Feature scaling

Logistic regression is sensitive to the scale of input features.

In this step, we:
- Identify numerical features that are not binary or categorical
- Apply `StandardScaler` to normalize those features

Note: Scaling is not strictly required for decision trees or ensembles, but we apply it for flexibility and consistency across modeling approaches.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Identify columns to scale: numerical
numeric_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Detect binary columns (0 or 1 only)
binary_cols = [col for col in df_encoded.columns
               if df_encoded[col].dropna().nunique() == 2 and set(df_encoded[col].unique()) <= {0, 1}]

# Exclude binary and other manually flagged columns
exclude = set(binary_cols + [
    'Attrition',  # target
    'HighIncomeFlag', 'LowSatisfactionFlag',
    'HighIncomeShortTenure', 'StressRisk'
])

# Keep only those that are continuous
scale_cols = [col for col in numeric_cols if col not in exclude]

# Initialize scaler
scaler = StandardScaler()

# Apply scaling
df_encoded[scale_cols] = scaler.fit_transform(df_encoded[scale_cols])

print(f"Scaled {len(scale_cols)} numeric features.")
print("Scaled columns:")
for col in scale_cols:
    print(f"  {col}")

### Histogram comparison: Before and after scaling

We plot the distribution of selected numeric features before and after scaling. This helps verify that scaling transformed the features to standard normal (mean ≈ 0, std ≈ 1), while preserving their shape and distribution.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Prepare original and scaled versions
df_unscaled = df.copy()
df_scaled = df_encoded.copy()

# Set up grid
num_features = len(scale_cols)
cols_per_row = 2  # Original vs scaled
rows = num_features

fig, axes = plt.subplots(rows, cols_per_row, figsize=(12, rows * 3))
sns.set(style='whitegrid')

for i, col in enumerate(scale_cols):
    # Original
    sns.histplot(df_unscaled[col], kde=True, ax=axes[i, 0], bins=30, color='skyblue')
    axes[i, 0].set_title(f"Original: {col}")
    
    # Scaled
    sns.histplot(df_scaled[col], kde=True, ax=axes[i, 1], bins=30, color='salmon')
    axes[i, 1].set_title(f"Scaled: {col}")

plt.tight_layout()
plt.suptitle("Before vs After Scaling (All Numeric Features)", fontsize=16, y=1.02)
plt.show()

## 7. Train-test split

We split the dataset into training and testing sets to evaluate generalization.

Key considerations:
- We use a stratified split on the target (`Attrition`) to preserve class balance.
- 80% of the data is used for training; 20% is held out for testing.
- Random seed is set for reproducibility.

In [ ]:
from sklearn.model_selection import train_test_split

# Define feature matrix and target
X = df_encoded.drop(columns='Attrition')
y = df_encoded['Attrition']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

# Confirm shape and class balance
print(f"Split complete:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_test:  {X_test.shape}, y_test:  {y_test.shape}")

print("\nClass distribution in y_train:")
display(y_train.value_counts(normalize=True).round(3))

print("Class distribution in y_test:")
display(y_test.value_counts(normalize=True).round(3))

## 8. Handling class imbalance

The target variable `Attrition` is imbalanced, with far more `No` than `Yes` cases.

To address this:
- We apply **SMOTE (Synthetic Minority Oversampling Technique)** to the training data.
- SMOTE generates synthetic examples of the minority class by interpolating between existing ones.
- We apply it **only to the training set** to avoid information leakage.

This step is useful when using models sensitive to class imbalance (e.g., logistic regression, SVM).

In [ ]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=RANDOM_STATE)

# Apply SMOTE to training data only
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Confirm result
print("SMOTE applied to training data.")
print(f"  X_train before: {X_train.shape}, after: {X_train_resampled.shape}")
print("\nClass distribution after SMOTE:")
display(y_train_resampled.value_counts(normalize=True).round(3))

## 9. Export preprocessed dataset

We export the resampled and original datasets to the `../data/processed/` directory for use in the modeling phase.

Saved files:
- `X_train.csv`, `y_train.csv` — raw stratified training set
- `X_test.csv`, `y_test.csv` — untouched test set
- `X_train_resampled.csv`, `y_train_resampled.csv` — SMOTE-balanced training set

In [ ]:
import os

# Create output directory if it doesn't exist
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Export each component safely
exports = {
    "X_train.csv": X_train,
    "y_train.csv": y_train,
    "X_test.csv": X_test,
    "y_test.csv": y_test,
    "X_train_resampled.csv": X_train_resampled,
    "y_train_resampled.csv": y_train_resampled
}

for filename, df_to_export in exports.items():
    try:
        full_path = os.path.join(output_dir, filename)
        df_to_export.to_csv(full_path, index=False)
        print(f"✅ Exported: {filename}")
    except Exception as e:
        print(f"❌ Failed to export {filename}: {e}")

# Preprocessing Summary and Next Steps

In this notebook, we:

- Reloaded and validated the cleaned dataset (`data_01.csv`)
- Performed final data integrity checks (nulls, constants, schema)
- Engineered domain-relevant features (e.g., tenure ratios, satisfaction flags)
- Encoded categorical variables using label encoding and one-hot encoding
- Scaled numeric features to standard normal
- Applied a stratified train-test split to preserve class distribution
- Handled class imbalance using SMOTE on the training set
- Exported all relevant datasets for downstream modeling

---

### Next Steps (Modeling Phase)
In the next notebook (`03_modeling.ipynb`), we will:

- Load the preprocessed datasets
- Train baseline classification models (Logistic Regression, Random Forest, etc.)
- Evaluate using accuracy, recall, precision, F1-score, and ROC AUC
- Tune hyperparameters and compare model performance
